In [ ]:
import pandas as pd
from bs4 import BeautifulSoup

file_path = 'all_toxic_comments.xml'

with open(file_path, 'r', encoding='utf-8') as file:
    data = file.read()

In [ ]:
soup = BeautifulSoup(data, 'xml')

texts = []
toxs = []
tox_rates = []
responses = []
tox_types = []
lexes = []
phrases = []
phrase_types = []

texts = data.split('</text>')

data_list = []

for text in texts:
    if '<text>' in text:

        text += '</text>'

        soup = BeautifulSoup(text, 'xml')

        text_content = soup.find('text').get_text(strip=True)

        for tox in soup.find_all('tox'):
            entry = {
                'text': text_content,
                'tox': tox.get_text(strip=True),
                'tox_rate': tox.get('rate', ''),
                'response': tox.get('response', ''),
                'tox_type': tox.get('type', ''),
                'phrases': [],
                'phrase_types': [],
                'lexes': [],
                'lex_counts': []
            }
            for phrase in tox.find_all('phrase'):
                phrase_text = phrase.get_text(strip=True)
                entry['phrases'].append(phrase_text)
                entry['phrase_types'].append(phrase.get('type', ''))

                lex_elements = phrase.find_all('lex')

                lex_texts = ', '.join([lex.get_text(strip=True) for lex in lex_elements])

                entry['lexes'].append(lex_texts)
                entry['lex_counts'].append(len(lex_elements))

            data_list.append(entry)

df = pd.DataFrame(data_list)

indexes = list(range(len(df)))
df['text_id'] = indexes

df = df.reindex(columns=['text_id', 'text', 'tox', 'tox_rate', 'response', 'tox_type', 'phrases', 'phrase_types', 'lexes', 'lex_counts'])

df_exploded = df.explode(['phrases', 'phrase_types', 'lexes', 'lex_counts'])

In [ ]:
df_exploded.to_csv('rutoxicon.csv', index=False, encoding='utf-8')

In [ ]:
len(df_exploded)

1609

In [ ]:
len(df)

1099

In [ ]:
df_exploded.isna().any()
df_exploded.[df_exploded.isna().any()].tolist()

AttributeError: 'DataFrame' object has no attribute 'rows'

In [ ]:
import pandas as pd
data = pd.read_csv('rutoxicon.csv')
data.head()


,text_id,text,tox,tox_rate,response,tox_type,phrases,phrase_types,lexes,lex_counts
0,0,Кармодрочеры,Кармодрочеры,4,post: animate,general_insult,Кармодрочеры,direct,Кармодрочеры,1
1,1,"Да блять ! 🤦 Он не взламывал игры, а симал их....","Да блять ! 🤦 Он не взламывал игры, а симал их....",5,post: animate,profanity,Да блять !,direct,блять,1
2,1,"Да блять ! 🤦 Он не взламывал игры, а симал их....","Да блять ! 🤦 Он не взламывал игры, а симал их....",5,post: animate,profanity,Это дохуя разные вещи,direct,дохуя,1
3,1,"Да блять ! 🤦 Он не взламывал игры, а симал их....","Да блять ! 🤦 Он не взламывал игры, а симал их....",5,post: animate,profanity,Так что про пирата - тут с ахуенной натяжкой.,direct,ахуенной,1
4,2,Вы там охуели что-ли за 6500 стрижка ? Это где...,Вы там охуели что-ли за 6500 стрижка ? Это где...,8,post: inanimate,profanity,Вы там охуели что-ли за 6500 стрижка,direct,охуели,1


In [ ]:
!pip install pymorphy3
import pymorphy3
from pymorphy3 import MorphAnalyzer

In [ ]:
!pip install pymystem3
from pymystem3 import Mystem


In [ ]:
m = Mystem()

Installing mystem to /root/.local/bin/mystem from http://download.cdn.yandex.net/mystem/mystem-3.1-linux-64bit.tar.gz


In [ ]:
MorphAnalyzer().parse('въебите')

[Parse(word='въебите', tag=OpencorporaTag('NOUN,inan,masc sing,loct'), normal_form='въебит', score=0.2524271844660194, methods_stack=((DictionaryAnalyzer(), 'бите', 247, 5), (UnknownPrefixAnalyzer(score_multiplier=0.5), 'въе'))),
 Parse(word='въебите', tag=OpencorporaTag('NOUN,inan,femn sing,datv'), normal_form='въебита', score=0.2524271844660194, methods_stack=((DictionaryAnalyzer(), 'бите', 462, 2), (UnknownPrefixAnalyzer(score_multiplier=0.5), 'въе'))),
 Parse(word='въебите', tag=OpencorporaTag('NOUN,inan,femn sing,loct'), normal_form='въебита', score=0.2524271844660194, methods_stack=((DictionaryAnalyzer(), 'бите', 462, 6), (UnknownPrefixAnalyzer(score_multiplier=0.5), 'въе'))),
 Parse(word='въебите', tag=OpencorporaTag('VERB,perf,tran plur,impr,excl'), normal_form='въести', score=0.21359223300970873, methods_stack=((FakeDictionary(), 'въебите', 1361, 14), (KnownSuffixAnalyzer(min_word_length=4, score_multiplier=0.5), 'ебите'))),
 Parse(word='въебите', tag=OpencorporaTag('NOUN,inan

In [ ]:
import re

In [ ]:
 m.analyze('ТП')[0]['analysis']

[]

In [ ]:
re.findall('[A-Z]*[A-Z]+?', )

['ADV']

In [ ]:
class Morph_Analyzer():
  def __init__(self):
    self.cash = {}
    self.morpho = Mystem()
  def get_POS(self, word):
    try:
      word = word.lower()
      try:
        pos = self.cash[word]
      except KeyError:
        try:
          gram =  self.morpho.analyze(word)[0]['analysis'][0]['gr']
          pos = ''.join(re.findall('[A-Z]*[A-Z]+?', gram))
        except IndexError:
          pos = ''
        except KeyError:
          pos = ''
        self.cash[word] = pos
    except AttributeError:
        print(word)
        pos = ''
    return pos


In [ ]:
new_data = data.dropna().reset_index()
new_data.drop('index', axis = 1)


,text_id,text,tox,tox_rate,response,tox_type,phrases,phrase_types,lexes,lex_counts
0,0,Кармодрочеры,Кармодрочеры,4,post: animate,general_insult,Кармодрочеры,direct,Кармодрочеры,1
1,1,"Да блять ! 🤦 Он не взламывал игры, а симал их....","Да блять ! 🤦 Он не взламывал игры, а симал их....",5,post: animate,profanity,Да блять !,direct,блять,1
2,1,"Да блять ! 🤦 Он не взламывал игры, а симал их....","Да блять ! 🤦 Он не взламывал игры, а симал их....",5,post: animate,profanity,Это дохуя разные вещи,direct,дохуя,1
3,1,"Да блять ! 🤦 Он не взламывал игры, а симал их....","Да блять ! 🤦 Он не взламывал игры, а симал их....",5,post: animate,profanity,Так что про пирата - тут с ахуенной натяжкой.,direct,ахуенной,1
4,2,Вы там охуели что-ли за 6500 стрижка ? Это где...,Вы там охуели что-ли за 6500 стрижка ? Это где...,8,post: inanimate,profanity,Вы там охуели что-ли за 6500 стрижка,direct,охуели,1
...,...,...,...,...,...,...,...,...,...,...
1348,1092,Въебите ей уже кто нибудь,Въебите ей уже кто нибудь,8,post: animate,threat,Въебите ей уже кто нибудь,direct,Въебите,1
1349,1093,И таким ипанутым на всю бошку истеричкам выдаю...,И таким ипанутым на всю бошку истеричкам выдаю...,6,post: animate,general_insult,И таким ипанутым на всю бошку истеричкам,direct,"ипанутым, истеричкам",2
1350,1094,Я думал там между машин ее зажало. А это очере...,Я думал там между машин ее зажало. А это очере...,3,post: animate,general_insult,А это очередная ТП мама меня фура убила.,indirect,ТП,1
1351,1096,"Пиздец , ну будут царапины и вмятины на машине...","Пиздец , ну будут царапины и вмятины на машине...",7,post: animate,profanity,"Пиздец ,",direct,Пиздец,1


In [ ]:
data_list = []
for index, row in new_data.iterrows():
  text = row['text']
  text_id = row['text_id']
  tox = row['tox']
  tox_rate = row['tox_rate']
  response = row['response']
  tox_type = row['tox_type']
  phrases = row['phrases']
  phrase_types = row['phrase_types']
  lex_counts = row['lex_counts']
  lexes = row['lexes']
  listik = lexes.split(', ')
  for l in listik:
    l = l.strip(' ')
    l = l.lower()
    entry = {'text_id': text_id, 'text':text, 'tox':tox, 'tox_rate': tox_rate,
             'response':response, 'tox_type':tox_type,
             'phrases': phrases,
             'phrase_types': phrase_types,
              'lex':l,
             'lex_counts': lex_counts}
    data_list.append(entry)



In [ ]:
cool_df = pd.DataFrame(data_list)

In [ ]:
cool_df = cool_df.reindex(columns=['text_id', 'text', 'tox', 'tox_rate', 'response', 'tox_type', 'phrases', 'phrase_types', 'lex', 'lex_counts'])
cool_df

,text_id,text,tox,tox_rate,response,tox_type,phrases,phrase_types,lex,lex_counts
0,0,Кармодрочеры,Кармодрочеры,4,post: animate,general_insult,Кармодрочеры,direct,кармодрочеры,1
1,1,"Да блять ! 🤦 Он не взламывал игры, а симал их....","Да блять ! 🤦 Он не взламывал игры, а симал их....",5,post: animate,profanity,Да блять !,direct,блять,1
2,1,"Да блять ! 🤦 Он не взламывал игры, а симал их....","Да блять ! 🤦 Он не взламывал игры, а симал их....",5,post: animate,profanity,Это дохуя разные вещи,direct,дохуя,1
3,1,"Да блять ! 🤦 Он не взламывал игры, а симал их....","Да блять ! 🤦 Он не взламывал игры, а симал их....",5,post: animate,profanity,Так что про пирата - тут с ахуенной натяжкой.,direct,ахуенной,1
4,2,Вы там охуели что-ли за 6500 стрижка ? Это где...,Вы там охуели что-ли за 6500 стрижка ? Это где...,8,post: inanimate,profanity,Вы там охуели что-ли за 6500 стрижка,direct,охуели,1
...,...,...,...,...,...,...,...,...,...,...
1878,1093,И таким ипанутым на всю бошку истеричкам выдаю...,И таким ипанутым на всю бошку истеричкам выдаю...,6,post: animate,general_insult,И таким ипанутым на всю бошку истеричкам,direct,ипанутым,2
1879,1093,И таким ипанутым на всю бошку истеричкам выдаю...,И таким ипанутым на всю бошку истеричкам выдаю...,6,post: animate,general_insult,И таким ипанутым на всю бошку истеричкам,direct,истеричкам,2
1880,1094,Я думал там между машин ее зажало. А это очере...,Я думал там между машин ее зажало. А это очере...,3,post: animate,general_insult,А это очередная ТП мама меня фура убила.,indirect,тп,1
1881,1096,"Пиздец , ну будут царапины и вмятины на машине...","Пиздец , ну будут царапины и вмятины на машине...",7,post: animate,profanity,"Пиздец ,",direct,пиздец,1


In [ ]:
obj = new_data.iloc[5]
#obj['lex'] = 'блять'
df = pd.DataFrame()
df.add(obj)
df.head()

""


In [ ]:
data.isna().any(axis=1)

,0
0,False
1,False
2,False
3,False
4,False
...,...
1605,False
1606,True
1607,True
1608,False


In [ ]:
poses = []
poses_dict = {}
for i in cool_df['lex']:
  pos = Morph_Analyzer().get_POS(i)
  poses_dict[i] = pos
  poses.append(pos)

In [ ]:
cool_df['Pos'] = cool_df['lex'].map(poses_dict)
cool_df

,text_id,text,tox,tox_rate,response,tox_type,phrases,phrase_types,lex,lex_counts,Pos
0,0,Кармодрочеры,Кармодрочеры,4,post: animate,general_insult,Кармодрочеры,direct,кармодрочеры,1,S
1,1,"Да блять ! 🤦 Он не взламывал игры, а симал их....","Да блять ! 🤦 Он не взламывал игры, а симал их....",5,post: animate,profanity,Да блять !,direct,блять,1,V
2,1,"Да блять ! 🤦 Он не взламывал игры, а симал их....","Да блять ! 🤦 Он не взламывал игры, а симал их....",5,post: animate,profanity,Это дохуя разные вещи,direct,дохуя,1,V
3,1,"Да блять ! 🤦 Он не взламывал игры, а симал их....","Да блять ! 🤦 Он не взламывал игры, а симал их....",5,post: animate,profanity,Так что про пирата - тут с ахуенной натяжкой.,direct,ахуенной,1,A
4,2,Вы там охуели что-ли за 6500 стрижка ? Это где...,Вы там охуели что-ли за 6500 стрижка ? Это где...,8,post: inanimate,profanity,Вы там охуели что-ли за 6500 стрижка,direct,охуели,1,V
...,...,...,...,...,...,...,...,...,...,...,...
1878,1093,И таким ипанутым на всю бошку истеричкам выдаю...,И таким ипанутым на всю бошку истеричкам выдаю...,6,post: animate,general_insult,И таким ипанутым на всю бошку истеричкам,direct,ипанутым,2,A
1879,1093,И таким ипанутым на всю бошку истеричкам выдаю...,И таким ипанутым на всю бошку истеричкам выдаю...,6,post: animate,general_insult,И таким ипанутым на всю бошку истеричкам,direct,истеричкам,2,S
1880,1094,Я думал там между машин ее зажало. А это очере...,Я думал там между машин ее зажало. А это очере...,3,post: animate,general_insult,А это очередная ТП мама меня фура убила.,indirect,тп,1,
1881,1096,"Пиздец , ну будут царапины и вмятины на машине...","Пиздец , ну будут царапины и вмятины на машине...",7,post: animate,profanity,"Пиздец ,",direct,пиздец,1,S


In [ ]:
cool_df.to_csv('rutoxicon_pos1.csv', index=False, encoding='utf-8')